In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
import numpy as np
import scanpy.api as sc
from anndata import read_h5ad
from anndata import AnnData
import scipy as sp
import scipy.stats
from gprofiler import GProfiler
import pickle
# Other specific functions 
from itertools import product
from statsmodels.stats.multitest import multipletests
import util
# R related packages 
import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
import anndata2ri

/home/martin/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython
# autoreload
%load_ext autoreload
%autoreload 2
# logging
sc.logging.print_versions()

scanpy==1.4.3 anndata==0.6.20 umap==0.3.8 numpy==1.16.4 scipy==1.2.1 pandas==0.25.0 scikit-learn==0.21.1 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
%%R
library(MAST)

## Load data

In [4]:
# Data path
data_path = '/data3/martin/tms_gene_data'
output_folder = data_path + '/DE_result'

In [5]:
# Load the data 
adata_combine = util.load_normalized_data(data_path)

In [6]:
temp_facs = adata_combine[adata_combine.obs['b_method']=='facs',]
temp_droplet = adata_combine[adata_combine.obs['b_method']=='droplet',]

## Generate a list of tissue-cell types for DE testing

In [7]:
cell_type_list = list(set(temp_droplet.obs['cell_ontology_class']))
tissue_list = list(set(temp_droplet.obs['tissue']))
min_cell_number = 100
analysis_list = []
analysis_info = {}
# for cell_type in cell_type_list:
for tissue,cell_type in product(tissue_list, cell_type_list):
    analyte = '%s.%s'%(tissue,cell_type)
    ind_select = (temp_droplet.obs['cell_ontology_class'] == cell_type) & \
                    (temp_droplet.obs['tissue'] == tissue)
    n_young = (temp_droplet.obs['age'][ind_select].isin(['1m', '3m'])).sum()
    n_old = (temp_droplet.obs['age'][ind_select].isin(['18m', '21m',
                                                   '24m', '30m'])).sum()
    analysis_info[analyte] = {}
    analysis_info[analyte]['n_young'] = n_young
    analysis_info[analyte]['n_old'] = n_old
    if (n_young>min_cell_number) & (n_old>min_cell_number) & (cell_type!='nan'):
        print('%s, n_young=%d, n_old=%d'%(analyte, n_young, n_old))
        analysis_list.append(analyte)

Heart_and_Aorta.endothelial cell of coronary artery, n_young=422, n_old=2526
Heart_and_Aorta.smooth muscle cell, n_young=110, n_old=263
Heart_and_Aorta.fibroblast of cardiac tissue, n_young=471, n_old=1901
Heart_and_Aorta.leukocyte, n_young=138, n_old=771
Thymus.double negative T cell, n_young=147, n_old=640
Thymus.thymocyte, n_young=696, n_old=1929
Thymus.DN4 thymocyte, n_young=238, n_old=1530
Spleen.B cell, n_young=5431, n_old=13845
Spleen.NK cell, n_young=159, n_old=264
Spleen.macrophage, n_young=482, n_old=1370
Spleen.T cell, n_young=1653, n_old=3473
Kidney.kidney cortex artery cell, n_young=153, n_old=275
Kidney.fenestrated cell, n_young=348, n_old=512
Kidney.kidney collecting duct principal cell, n_young=187, n_old=561
Kidney.podocyte, n_young=186, n_old=301
Kidney.kidney distal convoluted tubule epithelial cell, n_young=205, n_old=457
Kidney.kidney proximal convoluted tubule epithelial cell, n_young=1636, n_old=2813
Kidney.macrophage, n_young=191, n_old=1128
Kidney.kidney loop o

### DE using R package MAST 

In [8]:
## DE testing
gene_name_list = np.array(temp_droplet.var_names)
DE_result_MAST = {}
for i_analyte,analyte in enumerate(analysis_list):
    print(analyte, '%d/%d'%(i_analyte, len(analysis_list)))
    tissue,cell_type = analyte.split('.')
    ind_select = (temp_droplet.obs['cell_ontology_class'] == cell_type) & \
                    (temp_droplet.obs['tissue'] == tissue)
    adata_temp = temp_droplet[ind_select,]
    # reformatting
    adata_temp.X = np.array(adata_temp.X.todense())
    adata_temp.obs['condition'] = [int(x[:-1]) for x in adata_temp.obs['age']] 
    adata_temp.obs = adata_temp.obs[['condition', 'sex']]
    if len(set(adata_temp.obs['sex'])) <2:
        covariate = ''
    else:
        covariate = '+sex'
#     # toy example
#     covariate = ''
#     np.random.seed(0)
#     ind_select = np.random.permutation(adata_temp.shape[0])[0:100]
#     ind_select = np.sort(ind_select)
#     adata_temp = adata_temp[ind_select, 0:3]
#     adata_temp.X[:,0] = (adata_temp.obs['sex'] == 'male')*3
#     adata_temp.X[:,1] = (adata_temp.obs['condition'])*3
    # DE using MAST 
    R_cmd = util.call_MAST_age()
    get_ipython().run_cell_magic(u'R', u'-i adata_temp -i covariate -o de_res', R_cmd)
    de_res.columns = ['gene', 'raw-p', 'coef', 'bh-p']
    de_res.index = de_res['gene']
    DE_result_MAST[analyte] = pd.DataFrame(index = gene_name_list)
    DE_result_MAST[analyte] = DE_result_MAST[analyte].join(de_res)
    # fc between yound and old
    X = adata_temp.X
    y = (adata_temp.obs['condition']>10)
    DE_result_MAST[analyte]['fc'] = X[y,:].mean(axis=0) - X[~y,:].mean(axis=0)
#     break

Heart_and_Aorta.endothelial cell of coronary artery 0/62
Heart_and_Aorta.smooth muscle cell 1/62
Heart_and_Aorta.fibroblast of cardiac tissue 2/62
Heart_and_Aorta.leukocyte 3/62
Thymus.double negative T cell 4/62
Thymus.thymocyte 5/62
Thymus.DN4 thymocyte 6/62
Spleen.B cell 7/62
Spleen.NK cell 8/62
Spleen.macrophage 9/62
Spleen.T cell 10/62
Kidney.kidney cortex artery cell 11/62
Kidney.fenestrated cell 12/62
Kidney.kidney collecting duct principal cell 13/62
Kidney.podocyte 14/62
Kidney.kidney distal convoluted tubule epithelial cell 15/62
Kidney.kidney proximal convoluted tubule epithelial cell 16/62
Kidney.macrophage 17/62
Kidney.kidney loop of Henle thick ascending limb epithelial cell 18/62
Kidney.epithelial cell of proximal tubule 19/62
Bladder.bladder cell 20/62
Bladder.endothelial cell 21/62
Bladder.bladder urothelial cell 22/62
Lung.dendritic cell 23/62
Lung.mature natural killer T cell 24/62
Lung.capillary endothelial cell 25/62
Lung.intermediate monocyte 26/62
Lung.classical 

In [9]:
len(DE_result_MAST.keys())

62

### Save DE results

In [10]:
with open(output_folder+'/DE_tissue_cell_droplet.pickle', 'wb') as handle:
    pickle.dump(DE_result_MAST, handle)
    pickle.dump(analysis_list, handle)
    pickle.dump(analysis_info, handle)

### Validation

In [28]:
# Load DE result
with open(output_folder+'_old/DE_droplet.pickle', 'rb') as handle:
    DE_result_MAST_temp = pickle.load(handle)
    analysis_list_temp = pickle.load(handle)
for analyte in analysis_list:
    if analyte in analysis_list_temp:
        bh_p = DE_result_MAST[analyte]['bh-p']
        bh_p_temp = DE_result_MAST_temp[analyte]['bh-p']
        print('%s, New:%d, Old:%d, Overlap:%d'%(analyte, np.sum(bh_p<0.01), 
                                            np.sum(bh_p_temp<0.01), 
                                            np.sum((bh_p<0.01) & (bh_p_temp<0.01))))

Lung.fibroblast of lung, New:1106, Old:508, Overlap:478
Lung.capillary endothelial cell, New:1086, Old:1106, Overlap:1064
Liver.hepatocyte, New:5064, Old:1350, Overlap:1275
Spleen.T cell, New:6438, Old:6088, Overlap:5945
Spleen.B cell, New:4453, Old:4815, Overlap:4118
Limb_Muscle.mesenchymal stem cell, New:11837, Old:11950, Overlap:11785
Limb_Muscle.macrophage, New:4130, Old:2124, Overlap:2080
Limb_Muscle.T cell, New:689, Old:549, Overlap:489
Limb_Muscle.skeletal muscle satellite cell, New:5989, Old:6013, Overlap:5881
Limb_Muscle.endothelial cell, New:1653, Old:1699, Overlap:1563
Thymus.thymocyte, New:122, Old:131, Overlap:105
Marrow.granulocytopoietic cell, New:10373, Old:10065, Overlap:9909
Marrow.granulocyte, New:5549, Old:6094, Overlap:5290
Bladder.bladder cell, New:10150, Old:9587, Overlap:9547
Bladder.bladder urothelial cell, New:5571, Old:4007, Overlap:3623
